# Create and apply mask
## Part 3 of the series of notebooks to solve the competition


## Content:
  - [Import required modules](#Import%20required%20modules)
  - [load data](#load%20data)
  - [checking some special records](#checking%20some%20special%20records)
  - [Match length and strat index](#Match%20length%20and%20strat%20index )
  - [Mask](#Mask) 
  - [Saving masked data ](#Saving%20masked%20data)
  - [Un-mask](#Un-mask)

### Import required modules

In [2]:
# jupyter majec function to print images inlined
%matplotlib inline 

import sys
import numpy as np # linear algebra
import pandas as pd # data processing
from nltk.corpus import stopwords # load stoping words
from nltk.tokenize import word_tokenize # word tokenizer
import pickle # to save clean data

pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 100

import re # Regular expression

### load data

In [3]:
dataPath = 'dataset'

#Training data
train = pd.read_csv(dataPath+'/train.csv')
# Testing data 
test = pd.read_csv(dataPath+'/test.csv')

for col in train.columns:
    train[col] = train[col].astype(str)
for col in test.columns:
    test[col] = test[col].astype(str)

### checking some special records

In [4]:
train.shape

(27486, 4)

In [5]:
train.loc[[26788,26789]]

,textID,text,selected_text,sentiment
26788,7db6331c79,- I produce/direct/film/edit... I write... I coordinate events... I manage Don Fetti... there ain't much I don't do!,I write... I coordinate events... I manage Don Fetti... there ain't much I don't do!,neutral
26789,3b3670e7cf,having a great time with family!! big ups to my bro n sis in law for hanging out love n peace! {be free},great,positive


### Match length and strat index 

In [6]:
# this is old code, I liked to keep it
# match_length=[]
# start_index=[]
# for i in train.index:
#     start=train.text[i].find(train.selected_text[i])
#     start_index.append(start-1)
#     match_length.append(len(train.selected_text[i]))
#     if start<0:
#         print("error in ",i)

# train["match_length"]=match_length
# train["start_index"]=start_index

train["match_length"]=train.apply(lambda x : len(x.selected_text), axis=1)
train["start_index"]=train.apply(lambda x : x.text.find(x.selected_text), axis=1)

print("Done match_length Calc.")
train.head()

Done match_length Calc.


,textID,text,selected_text,sentiment,match_length,start_index
0,a3d0a7d5ad,"Spent the entire morning in a meeting w/ a vendor, and my boss was not happy w/ them. Lots of fun. I had other plans for my morning",my boss was not happy w/ them. Lots of fun.,neutral,43,55
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive,4,5
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! http://plurk.com/p/wxpdj,says good (or should i say bad?) afternoon!,neutral,43,0
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative,34,1
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive,6,6


### Mask

In [7]:
def mask(original, subtext, match_length, start_index):
    msk=np.zeros(len(original))
    msk[start_index:start_index+match_length]+=1
    return msk

def padding(lst):
    return np.append(lst, np.array([0]*(280-len(lst))))
    
masks = (train.apply(lambda x : mask(x.text, x.selected_text, x.match_length, x.start_index), axis=1)).apply(
        lambda x : padding(x))
train["masks"]=masks
train.head()

,textID,text,selected_text,sentiment,match_length,start_index,masks
0,a3d0a7d5ad,"Spent the entire morning in a meeting w/ a vendor, and my boss was not happy w/ them. Lots of fun. I had other plans for my morning",my boss was not happy w/ them. Lots of fun.,neutral,43,55,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...]"
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive,4,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! http://plurk.com/p/wxpdj,says good (or should i say bad?) afternoon!,neutral,43,0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative,34,1,"[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive,6,6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"


### Saving masked data 

In [9]:
def save(name,obj):
    pickleOut= open("dataset/pickled/"+name,"wb")
    pickle.dump(obj,pickleOut)
    pickleOut.close()

save ("masked_data",masks)

### Un-mask

In [62]:
def un_mask(string, mask): 
    return ''.join([string[i] for i in range(len(mask)) if int(mask[i]) == 1])

un_masked = train.apply(lambda x: un_mask(x.text,x.masks), axis=1)